In [1]:
import pandas as pd
import re
from pydicom.dataset import Dataset, FileDataset, FileMetaDataset
from pydicom.uid import generate_uid, ExplicitVRLittleEndian
from pydicom.sequence import Sequence
from datetime import datetime
from pathlib import Path
import pydicom
from dateutil import parser
from dateutil.tz import gettz

import base64
import cv2
import io
import numpy as np

import random
import matplotlib.pyplot as plt

In [4]:
ffn = r'C:\Users\dmattioli\OneDrive - University of Iowa\Downloads\Batch_5197022_batch_results.csv'

meta_cols_to_keep = ['HITId', 'HITTypeId', 'Reward', 'AssignmentId', 'WorkerId', 'AcceptTime', 'SubmitTime', 'WorkTimeInSeconds', 'Approve', 'Reject']
img_col_name = r'Answer.femurSemSeg.labeledImage.pngImageData'
meta_cols_to_keep.append(img_col_name)

# Define a dictionary to map timezone abbreviations to timezones
tzinfos = {
    'PST': gettz('America/Los_Angeles'),
    'PDT': gettz('America/Los_Angeles')
}

def import_mturk_csv(ffn: str) -> dict:
    df = pd.read_csv(ffn)

    # Drop any columns that are not in the meta_cols_to_keep list.
    df.drop(columns=[col for col in df.columns if col not in meta_cols_to_keep], inplace=True)

    # Name the table with the batch_id.
    df.name = re.search(r'Batch_(\d+)_batch_results.csv', ffn).group(1)

    # Convert the SubmitTime and AcceptTime columns to datetime using dateutil.parser with tzinfos
    df['SubmitTime'] = df['SubmitTime'].apply(lambda x: parser.parse(x, tzinfos=tzinfos))
    df['AcceptTime'] = df['AcceptTime'].apply(lambda x: parser.parse(x, tzinfos=tzinfos))

    # Create sub dataframes according to the grouped HITId values.
    grouped = df.groupby('HITId')
    sub_dfs = {hit_id: group.reset_index(drop=True) for hit_id, group in grouped}
    return sub_dfs


def decode_image(base64_str: str) -> np.ndarray:
    img_bytes = base64.b64decode(base64_str)
    img_array = np.frombuffer(img_bytes, dtype=np.uint8)
    img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (512, 512))
    return img


In [13]:
def create_dicom( subject_uid: str, user_uid: str, HIT_Id: str, data: pd.DataFrame ) -> Dataset: # bws: list, uids: dict ) -> Dataset:
        # Create a dicom file to represent the segmentations for the referenced dicom file
        file_meta = FileMetaDataset()
        # file_meta.MediaStorageSOPClassUID = self._mediastoragesopclassuid 
        # file_meta.MediaStorageSOPInstanceUID = pydicomUID.generate_uid()
        # file_meta.ImplementationClassUID = pydicomUID.generate_uid()
        # file_meta.TransferSyntaxUID = self._transfersyntaxuid
        file_meta.TransferSyntaxUID = ExplicitVRLittleEndian
        
        dcm = Dataset()
        dcm.StudyInstanceUID = subject_uid # to-do: should be pydicom compliant already, will need to come back to this with a revised metatable
        # HIT_id = str( uids['HITIDS'][0] )
        # dcm.UID = pydicomUID( HIT_id ) # HIT Id -- to-do: will need to redo this table to include pydicom compliant uids
        dcm.file_meta = file_meta
        # dcm.ReferencedSOPInstanceUID = original_dcm.SOPInstanceUID
        # dcm.InstanceCreationDate, dcm.InstanceCreationTime = self._datetime.date, self._datetime.time
        dcm.InstanceCreatorUID = user_uid
        dcm.ImageType = ['DERIVED', 'PRIMARY', 'SEGMENTATION']
        dcm.Modality = 'SEG' # Key Object Selection
        dcm.LargestImagePixelValue, dcm.SmallestImagePixelValue = np.uint( 1 ), np.uint( 0 )
        dcm.add_new( 0x00080050, 'LO', HIT_Id )  # Use LO VR for Accession Number
        dcm.SeriesDescription = 'Instructions Refinement - Iteration N' # TBD**********
        dcm.SegmentSequence = create_segment_sequence( data )
        dcm.is_little_endian = True
        dcm.is_implicit_VR = False
        return dcm


def create_segment_sequence( sub_df: pd.DataFrame ) -> Sequence: # uids: dict, bws: list ) -> Sequence:
    segment_sequence = Sequence()
    for idx, row in sub_df.iterrows():
        seg = Dataset()
        seg.SegmentNumber = idx # Worker ID here
        seg.OperatorsName = str( row['WorkerId'] ) # Worker ID here
        seg.add_new( (0x0019, 0x10a0), 'LO', "Operators' Name : WorkerID" )
        seg.SegmentLabel = str( row['AssignmentId'] ) # To:do: Assignment ID here to denote the unique submission of the worker for this HIT
        # seg.SegmentAlgorithmType, seg.SegmentAlgorithmName, seg.SegmentationType = _SegmentAlgorithmType, self._SegmentAlgorithmName, self._SegmentationType
        
        # Create a CodeSequence for the SegmentedPropertyCategoryCodeSequence attribute
        code_sequence = Dataset()
        # code_sequence.CodeValue, code_sequence.CodingSchemeDesignator, code_sequence.CodeMeaning = self._object_type_code, self._CodingSchemeDesignator, self._object_type
        seg.SegmentedPropertyCategoryCodeSequence = Sequence( [code_sequence] )
        
        # Create a CodeSequence for the SegmentedPropertyTypeCodeSequence attribute -- same as above
        seg.SegmentedPropertyTypeCodeSequence = Sequence( [code_sequence.copy()] )
        
        # seg.SegmentDescription = 'Instructions Refinement - Iteration N'
        bw = decode_image( row[img_col_name] )
        seg.PixelData, seg.Rows, seg.Columns = bw.tobytes(), bw.shape[0], bw.shape[1]
        segment_sequence.append( seg )
    #     self._metatables.add_new_item( table_name='MTURK_WORKER_IDS', item_name=seg.SegmentNumber, extra_columns_values={'MTURK_HIT_ID': HIT_id } )
    #     self._metatables.add_new_item( table_name='MTURK_ASSIGNMENT_IDS', item_name=seg.SegmentLabel, extra_columns_values={'MTURK_HIT_ID': HIT_id, 'MTURK_WORKER_ID': seg.SegmentNumber } )
    # self._metatables.add_new_item( table_name='MTURK_HIT_IDS', item_name=HIT_id )
    return segment_sequence



# For each file in dicom_datasets, write as a dicom file to pn, storing the file names
file_names=[]
pn = r'C:\Users\dmattioli\OneDrive - University of Iowa\Downloads\tmp\example_mturk_compiled_data'
tst = import_mturk_csv(ffn)
for key, value in tst.items():
    dcm = create_dicom( subject_uid=generate_uid(), user_uid=generate_uid(), HIT_Id=key, data=value )
    fn = Path(pn) / f'{key}'
    file_names.append(fn)
    dcm.save_as(fn)

# Pick a random filename to dicomread and display the image stored.
fn = random.choice(file_names)
ds = pydicom.dcmread(fn)
img = ds.pixel_array
plt.imshow(img)
plt.show()

TypeError: object of type 'numpy.uint32' has no len()

In [ ]:
def create_dicom_from_mturk(HITs: dict) -> list:
    dicom_datasets = []

    for hit_id, df in HITs.items():
        # Create a new DICOM dataset object
        dicom_uid = generate_uid()
        file_meta = Dataset()
        file_meta.MediaStorageSOPClassUID = '1.2.840.10008.5.1.4.1.1.88.11'  # Comprehensive SR Storage
        file_meta.MediaStorageSOPInstanceUID =dicom_uid
        file_meta.ImplementationClassUID = generate_uid()
        file_meta.TransferSyntaxUID = ExplicitVRLittleEndian

        ds = FileDataset(fn, {}, file_meta=file_meta, preamble=b"\0" * 128)
        ds.SOPClassUID = '1.2.840.10008.5.1.4.1.1.88.11'  # Comprehensive SR Storage
        ds.SOPInstanceUID = dicom_uid
        # ds.StudyInstanceUID = dicom_uid # need to derive this from the existing xnat server -- should be the uid we used for the uploaded source images.
        # ds.SeriesInstanceUID = dicom_uid # same.
        ds.Modality = 'SR'
        # ds.SeriesNumber = 1
        # ds.InstanceNumber = 1
        ds.PatientID = 'REDACTED'
        ds.PatientName = 'REDACTED'
        ds.PatientBirthDate = '19000101'
        ds.PatientSex = 'O'
        date_time = df['SubmitTime'].iloc[0]
        ds.ContentDate = date_time.strftime('%Y%m%d')
        ds.ContentTime = date_time.strftime('%H%M%S')
        # ds.AccessionNumber = hit_id # hit_id is too long and is throwing an error (length 30 but max is 16 for VR SH)
        ds.add_new(0x00080050, 'LO', hit_id)  # Use LO VR for Accession Number
        # print( len(hit_id))


        # Add the metadata from the DataFrame to the DICOM dataset
        image_data = []
        for _, row in df.iterrows():
            # Create a content item for each row
            content_item = Dataset()
            content_item.ValueType = 'TEXT'
            content_item.add_new(0x0040A160, 'UT', f"WorkerId: {row['WorkerId']}, Reward: {row['Reward']}, AssignmentId: {row['AssignmentId']}")
            content_item.ConceptNameCodeSequence = Sequence([Dataset()])
            content_item.ConceptNameCodeSequence[0].CodeValue = '121071'
            content_item.ConceptNameCodeSequence[0].CodingSchemeDesignator = 'DCM'
            content_item.ConceptNameCodeSequence[0].CodeMeaning = 'Textual description'
            ds.add_new(0x0040A730, 'SQ', Sequence([content_item]))

            # extract the encoded image string.
            image_data.append( decode_image( row[img_col_name] ) )
        
        # Combine all images into an NxRxC array; store the combined image in the DICOM dataset.
        combined_image = np.stack(image_data, axis=0)
        ds.Rows, ds.Columns, ds.SamplesPerPixel = combined_image.shape[1], combined_image.shape[2], combined_image.shape[3]
        ds.PixelRepresentation = 0  # 0 for unsigned integers
        ds.BitsStored = 8
        ds.HighBit = 7
        ds.PhotometricInterpretation, ds.BitsAllocated = 'RGB', 8
        ds.PixelData = combined_image.tobytes()
        ds.is_little_endian = True
        ds.is_implicit_VR = False

        # print(f"{'---'*50}\n{ds}\n{'---'*50}\n")
        dicom_datasets.append(ds)

    return dicom_datasets

poo = import_mturk_csv(ffn)
dicom_datasets = create_dicom_from_mturk(poo)

pn = r'C:\Users\dmattioli\OneDrive - University of Iowa\Downloads\tmp\example_mturk_compiled_data'
# For each file in dicom_datasets, write as a dicom file to pn, storing the file names
file_names=[]
for i, ds in enumerate(dicom_datasets):
    # Filename will be the dicom's accessionNumber for now
    fn = Path(pn) / f'{ds.AccessionNumber}'
    file_names.append(fn)
    ds.save_as(fn)

# Pick a random filename to dicomread and display the image stored.
fn = random.choice(file_names)
ds = pydicom.dcmread(fn)
img = ds.pixel_array
plt.imshow(img)
plt.show()

### Older code:

In [ ]:
class OtherDicomSession():
    def __init__( self, batch_ffn: str, login: XNATLogin, xnat_connection: XNATConnection, metatables: MetaTables, object: str, print_out: bool=False ):
        self._batch_ffn = batch_ffn
        self._login = login
        self._xnat_connection = xnat_connection
        self._metatables = metatables
        assert self._metatables.item_exists( table_name='Acquisition_sites', item_name='AMAZON_MECHANICAL_TURK' )
        assert self._metatables.table_exists( 'Segmentation_Objects' )
        assert self._metatables.item_exists( table_name='Segmentation_Objects', item_name=object )
        self._acquisition_site, self._object = 'AMAZON_MECHANICAL_TURK', object

        
        # metatables.add_new_item( table_name='MTURK_BATCH_IDS', item_name='Segmentation', extra_columns_values={'task':'Segmentation'} )


        # attributes that metatables will eventually need to store -- to-do:
        self._object, self._object_type = object, 'BONE'
        self._object_code, self._object_type_code, self._CodingSchemeDesignator = 'T-12760', 'T-32000', 'SRT' # Code for 'Femur', 'Bone', according to copilot gpt
        self._SegmentAlgorithmType, self._SegmentAlgorithmName, self._SegmentationType = 'MANUAL', 'MTURK', 'BINARY'
        
        self._mediastoragesopclassuid, self._transfersyntaxuid = pydicomUID('1.2.840.10008.5.1.1.4'), pydicomUID('1.2.840.10008.1.2') # Basic Grayscale Image Box SOP Class, Implicit VR Little Endian
        self._print_out = print_out    

        # Create a new df with the number of rows equal to the number of groups. create a column for the respective Input.image_url
        self._raw_data = pd.read_csv( self._batch_ffn )
        self._datetime = USCentralDateTime( self._raw_data.loc[0, 'ApprovalTime'] )
        icol = self._get_pngImageData_col()
        self._raw_data['BinaryImages'] = [self.convert_base64_to_np_array( row[icol] ) for i, row in self._raw_data.iterrows()]
        self._grouped_data = self._raw_data.groupby( 'Input.image_url' )
        self._df = pd.DataFrame( index=range( len( self._grouped_data ) ), columns=['SUBJECT_XNAT_UID',
                                                                                    'image_hashes_mt_uid', 'subject_mt_uid',
                                                                                    'FN', 'IS_VALID', 'IMAGE_HASHES', 'BWS' 'DCMDATA', 'UID_INFO', 'FFN'] )
        
        # for each row of self._df compute the ImageHash for the s3url in that row's Image
        self._df['FFN'] = [group for group, data in self._grouped_data]
        self._df['FN'] = [s3url.split('/')[-1] for s3url in self._df['FFN']]
        self._df['IMAGE_HASHES'] = [self._read_image( s3url ) for s3url in self._df['FFN']]
        self._df['UID_INFO'] = self._grouped_data.apply(lambda g: {'HITIDS': g['HITId'].tolist(), 
                                                            'ASSIGNMENTIDS': g['AssignmentId'].tolist(), 
                                                            'WORKERIDS': g['WorkerId'].tolist()}).reset_index(drop=True)
        
        # Store a list of the binary images in the BWS col
        self._df['BWS'] = [data['BinaryImages'].to_list() for group, data in self._grouped_data]

        # given that hash_str is a property of each item in the objects stored in each row of self._df['ImageHashes'], apply self._metatables.item_exists( table_name='IMAGE_HASHES', item_name=hash_str) via list comprehension
        self._df['IS_VALID'] = [self._metatables.item_exists(table_name='IMAGE_HASHES', item_name=item.hash_str) for item in self._df['IMAGE_HASHES']]

        # Create a dicom file for each row such that the semantic segmentation data is stacked
        self._create_DCMDATA()

        # create a list of dataframes such that each dataframe groups together the self._df rows that correspond to images belonging to the same subject.
        self._group_rows_by_subject()
        
        
    def _get_table( self, table_name: str ) -> pd.DataFrame: #to-do: move this to metatables class definition.
        assert self._metatables.table_exists( table_name ), f'Table {table_name} does not exist in metatables'
        return self._metatables.tables[table_name]

    # def _get_item( self, table_name: str, item_name: str ) -> pd.Series: #to-do: move this to metatables class definition
    #     # table name represents the table in self._metatables.tables that we want. item_name is the row of that table that we want. return the row as a pd.series
    #     return self._get_table(table_name).loc[item_name]


    def _read_image( self, s3url: str ) -> ImageHash:
        response = requests.get( s3url, stream=True )
        response.raw.decode_content = True
        arr = np.asarray( bytearray( response.raw.read() ), dtype=np.uint8 )
        return ImageHash( metatables=self._metatables, img=cv2.imdecode( arr, cv2.IMREAD_GRAYSCALE ) ) # img = Image.open( response.raw )

    # write a method that returns the column name containing the substring .pngImageData
    def _get_pngImageData_col( self ) -> str:
        return [c for c in self._raw_data.columns if '.pngImageData' in c][0]

    def convert_base64_to_np_array( self, b64_str: str ) -> np.ndarray:
        return cv2.imdecode( np.frombuffer( base64.b64decode( b64_str ), np.uint8 ), cv2.IMREAD_GRAYSCALE )

    def _derive_subject_instance_num( self, subject_xnat_uid: str, hash_str: str, image_hashes_mt_uid: str ) -> str: # filename that we need to mimic
            
        pass

    def _create_DCMDATA( self ):
        hash_table, subject_table, registered_user_table = self._get_table( 'IMAGE_HASHES' ), self._get_table( 'SUBJECTS' ), self._get_table( 'REGISTERED_USERS' )
        user_mt_uid = registered_user_table.loc[registered_user_table['NAME'] == self._login.validated_username.upper(), 'UID'].values[0]
        for i, row in self._df.iterrows():
            if not row['IS_VALID']:
                continue
            image_hashes_mt_uid = row['IMAGE_HASHES'].hash_str.upper()
            subject_mt_uid = hash_table.loc[hash_table['NAME'] == image_hashes_mt_uid, 'SUBJECT'].values[0] # type: ignore
            self._df.loc[i,'SUBJECT_XNAT_UID'] = subject_table.loc[subject_table['UID'] == subject_mt_uid, 'NAME'].values[0] # type: ignore
            self._df.loc[i,'image_hashes_mt_uid'] = image_hashes_mt_uid
            self._df.loc[i,'subject_mt_uid'] = subject_mt_uid
            dcm = self._create_dicom( subject_name=self._df.loc[i,'SUBJECT_XNAT_UID'], user_uid=user_mt_uid, bws=row['BWS'], uids=row['UID_INFO'] )
            self._df.loc[i, 'DCMDATA'] = [dcm]

    def _create_dicom( self, subject_name: str, user_uid: str, bws: list, uids: dict ) -> pydicomDataset:
        # Create a dicom file to represent the segmentations for the referenced dicom file
        file_meta = pydicomFileMetaDataset()
        file_meta.MediaStorageSOPClassUID = self._mediastoragesopclassuid 
        # file_meta.MediaStorageSOPInstanceUID = pydicomUID.generate_uid()
        # file_meta.ImplementationClassUID = pydicomUID.generate_uid()
        file_meta.TransferSyntaxUID = self._transfersyntaxuid
        
        dcm = pydicom.Dataset()
        dcm.StudyInstanceUID = pydicomUID( subject_name ) # to-do: should be pydicom compliant already, will need to come back to this with a revised metatable
        HIT_id = str( uids['HITIDS'][0] )
        # dcm.UID = pydicomUID( HIT_id ) # HIT Id -- to-do: will need to redo this table to include pydicom compliant uids
        dcm.file_meta = file_meta
        # dcm.ReferencedSOPInstanceUID = original_dcm.SOPInstanceUID
        dcm.InstanceCreationDate, dcm.InstanceCreationTime = self._datetime.date, self._datetime.time
        dcm.InstanceCreatorUID = generate_pydicomUID() # to-do: will need to redo this table to include pydicom compliant uids
        # dcm.InstanceCreatorUID = user_uid # to-do: will need to redo this table to include pydicom compliant uids
        dcm.ImageType = ['DERIVED', 'PRIMARY', 'SEGMENTATION']
        dcm.Modality = 'KO' # Key Object Selection
        dcm.LargestImagePixelValue, dcm.SmallestImagePixelValue = np.uint( 1 ), np.uint( 0 )
        dcm.SeriesDescription = 'Instructions Refinement - Iteration N'
        dcm.SegmentSequence = self._create_segment_sequence( uids, bws )
        return dcm

    def _create_segment_sequence( self, uids: dict, bws: list ) -> Sequence:
        segment_sequence = Sequence()
        for idx, bw in enumerate( bws ):
            seg = Dataset()
            seg.SegmentNumber = idx # Worker ID here
            seg.OperatorsName = str( uids['WORKERIDS'][idx] ) # Worker ID here
            seg.add_new( (0x0019, 0x10a0), 'LO', "Operators' Name : WorkerID" )
            seg.SegmentLabel = str( uids['ASSIGNMENTIDS'][idx] ) # To:do: Assignment ID here to denote the unique submission of the worker for this HIT
            seg.SegmentAlgorithmType, seg.SegmentAlgorithmName, seg.SegmentationType = self._SegmentAlgorithmType, self._SegmentAlgorithmName, self._SegmentationType
            
            # Create a CodeSequence for the SegmentedPropertyCategoryCodeSequence attribute
            code_sequence = Dataset()
            code_sequence.CodeValue, code_sequence.CodingSchemeDesignator, code_sequence.CodeMeaning = self._object_type_code, self._CodingSchemeDesignator, self._object_type
            seg.SegmentedPropertyCategoryCodeSequence = Sequence( [code_sequence] )
            
            # Create a CodeSequence for the SegmentedPropertyTypeCodeSequence attribute -- same as above
            seg.SegmentedPropertyTypeCodeSequence = Sequence( [code_sequence.copy()] )
            
            # seg.SegmentDescription = 'Instructions Refinement - Iteration N'
            seg.PixelData, seg.Rows, seg.Columns = bw.tobytes(), bw.shape[0], bw.shape[1]
            segment_sequence.append( seg )
        #     self._metatables.add_new_item( table_name='MTURK_WORKER_IDS', item_name=seg.SegmentNumber, extra_columns_values={'MTURK_HIT_ID': HIT_id } )
        #     self._metatables.add_new_item( table_name='MTURK_ASSIGNMENT_IDS', item_name=seg.SegmentLabel, extra_columns_values={'MTURK_HIT_ID': HIT_id, 'MTURK_WORKER_ID': seg.SegmentNumber } )
        # self._metatables.add_new_item( table_name='MTURK_HIT_IDS', item_name=HIT_id )
        return segment_sequence
    
    def _group_rows_by_subject( self ):
        pass


    def write( self, idcm: int, zip_dest: Opt[str] = None, print_out: Opt[bool] = False ) -> str: #write individiual dicom files to a zipped folder
        
        if not self._df.loc[idcm, 'IS_VALID']:
            # if print_out:
            print( f'***Session is invalid; could be for several reasons. try evaluating whether all of the image hash_strings already exist in the matatable.' )
        pass

    def catalog_new_data( self ):
        self._metatables.save( print_out=print_out )
        pass

batch_ffn = r'C:\Users\dmattioli\Projects\XNAT\data\examples\MTurkSemanticSegmentation_Example_2.csv'
poo = OtherDicomSession( batch_ffn, validated_login, connection, metatables, object='Humerus' )


In [ ]:
class MTurkSemanticSegmentation( ScanFile ):
    ''' # Example usage:
    print( MTurkSemanticSegmentation( pd.read_csv( r'...\\data\\examples\\MTurkSemanticSegmentation_Example_File.csv' ) ) )
    '''
    def __init__( self, assignment: pd.Series, metatables: MetaTables ): #to-do: allow for different input types eg batch file data or pulled-from-xnat data
        super().__init__( metatables )  # Call the __init__ method of the base class
        self._validate_input( assignment )
        self._read_image()
        self._extract_target_object_info() #to-do
        self._extract_date_and_time()
        # self._extract_uid_info()
        self._extract_pngImageData()
        self._check_data_validity()

    @property
    def bw( self ) -> np.ndarray:   return self._bw
    
    def _validate_input( self, assignment: pd.Series ):
        assert len( set(self.mturk_batch_col_names) - set(assignment.columns) ) == 0, f"Missing required columns: {set(self.mturk_batch_col_names) - set(assignment.columns)}"
        self._metadata = assignment.loc[0]
        img_s3_url = assignment.loc[0,'Input.image_url']
        assert self.is_s3_url( img_s3_url ), f'Input.image_url column of inputted data series (row) must be an s3 url: {img_s3_url}'
        self._ffn = self.metadata['Input.image_url']
        self._bw, self._acquisition_site = self.image.dummy_image(), 'AMAZON_MECHANICAL_TURK' #to-do: this is copy-pasted from the MetaTables, need to figure out how to query it.

    def _check_data_validity( self ): # need to check that the pnd image *does* exist in metatables
        self._is_valid = self.image.in_img_hash_metatable and not self.is_similar_to_template_image()
    
    def _extract_target_object_info( self ):
        pass

    def _extract_date_and_time( self ):
        self._datetime = USCentralDateTime( self.metadata.loc['SubmitTime'] )

    def _extract_pngImageData( self ):
        pngImageData_index = [i for i, c in enumerate( self.metadata.index.to_list() ) if '.pngImageData' in c]
        self._bw = self.convert_base64_to_np_array( self.metadata.iloc[pngImageData_index[0]] )

     
    def __str__( self ):
        return f'{self.__class__.__name__}:\t{self.ffn}\nIs Valid:\t{self.is_valid}\nAcquisition Site: {self.acquisition_site}\nGroup:\t\t{self.group}\nDatetime:\t{self.datetime}\nUID Info: {self.uid_info}'